In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
MIST v1.2 isochrones (non-rotating) three-parameter fit:
  (T_eff, log g, [Fe/H]) -> mass / radius / age (with a lower-age prior for reporting)

References:
- MIST User Guide & README_tables (column names and formats)
- MIST resources: read_mist_models.py helpers
Docs:
- https://waps.cfa.harvard.edu/MIST/README_overview.pdf
- https://waps.cfa.harvard.edu/MIST/README_tables.pdf
- https://waps.cfa.harvard.edu/MIST/resources.html
"""

import re
import numpy as np
import pandas as pd


# =========================
# 1) User inputs
# =========================
# Replace the three .iso file paths below with your local paths
files = [
    ('MIST_v1.2_feh_m0.25_afe_p0.0_vvcrit0.0_basic.iso', -0.25),
    ('MIST_v1.2_feh_p0.00_afe_p0.0_vvcrit0.0_basic.iso',  0.00),
    ("MIST_v1.2_feh_p0.25_afe_p0.0_vvcrit0.0_basic.iso", +0.25),
]

# Observational constraints (mean ± 1σ)
teff, sig_teff = 7724.0, 36.0
logg, sig_logg = 4.087, 0.05
feh_obs, sig_feh = -0.03, 0.03


# =========================
# 2) Readers and utilities
# =========================
def read_mist_iso(fn: str, feh_value: float) -> pd.DataFrame:
    """
    Read a MIST .iso file:
      - Parse the last '#' header line to obtain column names
      - Read the data with those names (header=None, comment='#')
      - Attach the file [Fe/H] value as 'feh_file'
    """
    header_line = None
    with open(fn, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            if line.startswith('#'):
                header_line = line.strip()
            else:
                break
    if header_line is None:
        raise RuntimeError(f"No header line starting with '#' found in {fn}")

    cols = re.split(r'\s+', header_line.lstrip('#').strip())
    cols = [c.strip().strip(',') for c in cols if c.strip()]

    df = pd.read_csv(
        fn, sep=r"\s+", header=None, names=cols,
        comment="#", engine="python"
    )
    df["feh_file"] = feh_value
    return df


def pick_col(df: pd.DataFrame, candidates) -> str | None:
    """Return the first existing column name from candidates, or None."""
    for c in candidates:
        if c in df.columns:
            return c
    return None


def weighted_quantiles(x, w, qs=(0.16, 0.50, 0.84)):
    """
    Weighted quantiles via sorting x and linearly interpolating the cumulative
    weight distribution. Returns array of same length as qs.
    """
    x = np.asarray(x, dtype=float)
    w = np.asarray(w, dtype=float)
    m = np.isfinite(x) & np.isfinite(w) & (w > 0)
    if not np.any(m):
        raise ValueError("All weights are zero or non-finite.")
    x, w = x[m], w[m]
    idx = np.argsort(x)
    x, w = x[idx], w[idx]
    cdf = np.cumsum(w)
    cdf /= cdf[-1]
    qs = np.atleast_1d(qs).astype(float)
    v = np.interp(qs, cdf, x)
    return v if len(qs) > 1 else v[0]


# =========================
# 3) Load and unify columns
# =========================
dfs = [read_mist_iso(fn, feh) for fn, feh in files]
df = pd.concat(dfs, ignore_index=True)

col_logTeff = pick_col(df, ["logTeff", "log_Teff", "log_Teffs", "log_TeffK"])
col_logg    = pick_col(df, ["logg", "log_g", "log_surface_gravity"])
col_mass    = pick_col(df, ["star_mass", "initial_mass", "mass", "Minit", "M"])
col_logR    = pick_col(df, ["logR", "log_R", "log_Radius"])
col_radius  = pick_col(df, ["radius", "R"])
col_logL    = pick_col(df, ["logL", "log_L", "log_Luminosity"])
# age sometimes given as log10(age/yr), sometimes as age(yr)
col_logage  = pick_col(df, ["log10_isochrone_age_yr", "log10_age", "log_age", "logAge"])
col_age     = pick_col(df, ["age", "isochrone_age_yr", "age_yr"])

missing = [n for n, c in {"logTeff": col_logTeff, "logg": col_logg, "mass": col_mass}.items() if c is None]
if missing:
    raise KeyError(f"Missing required columns: {missing}\n"
                   f"Columns example: {list(df.columns)[:20]} ...")

# Radius: prefer 'radius'; else use 10^logR; else derive from logL and Teff (L ∝ R^2 T^4)
if col_radius is None:
    if col_logR is None:
        if col_logL is not None:
            Teff_model_tmp = 10.0 ** df[col_logTeff].to_numpy()
            df["_radius_calc_"] = np.sqrt(10.0 ** df[col_logL].to_numpy()) / (Teff_model_tmp / 5772.0) ** 2
            col_radius = "_radius_calc_"
        else:
            raise KeyError("Neither 'radius' nor 'logR'/'logL' found; cannot derive radius.")
    else:
        df["_radius_calc_"] = 10.0 ** df[col_logR].to_numpy()
        col_radius = "_radius_calc_"


# =========================
# 4) Likelihood and weights (including [Fe/H])
# =========================
Teff_model = 10.0 ** df[col_logTeff].to_numpy()
logg_model = df[col_logg].to_numpy()
feh_model  = df["feh_file"].to_numpy()

chi2 = ((Teff_model - teff) / sig_teff) ** 2 \
     + ((logg_model - logg) / sig_logg) ** 2 \
     + ((feh_model  - feh_obs) / sig_feh) ** 2

w = np.exp(-0.5 * chi2)
w[np.isnan(w)] = 0.0
S = w.sum()
if S <= 0 or not np.isfinite(S):
    raise RuntimeError("All weights are zero or non-finite: "
                       "check [Fe/H] grid coverage or enlarge uncertainties.")
w /= S  # normalize


# =========================
# 5) Report mass and radius (weighted quantiles)
# =========================
q025, q16, q50, q84, q975 = weighted_quantiles(
    df[col_mass].to_numpy(), w, (0.025, 0.16, 0.50, 0.84, 0.975)
)

print(
    "MIST (rot/non-rot depends on files) mass quantiles: "
    f"68%: q16={q16:.3f}, q84={q84:.3f}; "
    f"95%: q2.5={q025:.3f}, q97.5={q975:.3f} (M_sun)"
)

print(
    f"median = {q50:.3f} "
    f"(68%: -{q50 - q16:.3f}/+{q84 - q50:.3f}; "
    f"95%: -{q50 - q025:.3f}/+{q975 - q50:.3f}) M_sun"
)

r16, r50, r84 = weighted_quantiles(df[col_radius].to_numpy(), w, (0.16, 0.50, 0.84))
print(f"MIST radius = {r50:.3f} (-{r50 - r16:.3f}/+{r84 - r50:.3f}) R_sun")


# =========================
# 6) Age posterior: aggregate by unique isochrone ages, then apply lower-age prior
# =========================
age_gyr_all = None

if col_logage is not None:
    logage = df[col_logage].to_numpy()            # log10(age/yr)
    age_gyr_all = (10.0 ** logage) / 1e9
elif col_age is not None:
    age_gyr_all = df[col_age].to_numpy() / 1e9

if age_gyr_all is None:
    raise RuntimeError("No age column found (neither log-age nor age in years).")

# (i) aggregate: sum weights for identical ages to avoid over-counting the same isochrone
age_key = np.round(age_gyr_all, 5)  # merge tiny numeric jitter
age_df = pd.DataFrame({"age": age_key, "w": w})
agg = age_df.groupby("age", as_index=False)["w"].sum().sort_values("age")
ages = agg["age"].to_numpy()
w_age = agg["w"].to_numpy()
w_age = w_age / w_age.sum()

# (ii) apply a lower-age prior and report ONLY this prior-applied result
age_min_gyr = 0.10  # recommended 0.10–0.20 Gyr
mask_prior = (ages >= age_min_gyr)
if np.any(mask_prior):
    ages_prior = ages[mask_prior]
    w_age_prior = w_age[mask_prior]
    w_age_prior = w_age_prior / w_age_prior.sum()

    def wq_1d(x, wx, qs=(0.16, 0.50, 0.84)):
        order = np.argsort(x)
        x, wx = x[order], wx[order]
        cdf = np.cumsum(wx); cdf /= cdf[-1]
        qs = np.clip(np.array(qs, float), 1e-12, 1 - 1e-12)
        return np.interp(qs, cdf, x)

    a16p, a50p, a84p = wq_1d(ages_prior, w_age_prior, (0.16, 0.50, 0.84))
    print(
        f"[Age + prior t>={age_min_gyr:.2f} Gyr] "
        f"age = {a50p:.3f} (-{a50p - a16p:.3f}/+{a84p - a50p:.3f}) Gyr"
    )
else:
    print(
        f"[Note] The lower-age prior t>={age_min_gyr:.2f} Gyr removes all posterior mass; "
        "weights are concentrated at younger ages. Consider checking uncertainties or relaxing the prior."
    )

C:\Users\pc\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


MIST (rot/non-rot depends on files) mass quantiles: 68%: q16=1.698, q84=1.795; 95%: q2.5=1.645, q97.5=1.834 (M_sun)
median = 1.744 (68%: -0.047/+0.050; 95%: -0.099/+0.089) M_sun
MIST radius = 2.043 (-0.228/+0.137) R_sun
[Age + prior t>=0.10 Gyr] age = 0.923 (-0.087/+0.052) Gyr
